 <a name="1"></a>
# <span style="color:green">Исследование надёжности заёмщиков <span>


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество
детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о
платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной
системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


# 1. Импорт библиотек и чтение данных

In [548]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.express as px
from plotly import tools
from plotly.offline import init_notebook_mode, iplot

In [549]:
df = pd.read_csv('datasets/preprocessing_project.csv')
df_copy = df.copy(deep=True)
df.head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья


# 2. Проверка данных и предварительный анализ



In [550]:
df.shape

(21525, 12)

In [551]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Информация о столбцах

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

In [552]:
df.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


In [553]:
df.duplicated().sum()

54

In [554]:
df.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12628,0,348335.804544,68,среднее,1,женат / замужем,0,F,пенсионер,0,39293.090380,покупка жилья для сдачи
11432,2,-4275.174149,31,высшее,0,женат / замужем,0,M,сотрудник,0,156259.268219,ремонт жилью
17907,0,336757.291998,59,среднее,1,гражданский брак,1,F,пенсионер,1,87485.120977,свадьба
16968,0,-1238.560080,52,среднее,1,женат / замужем,0,F,компаньон,0,351157.460749,покупка жилой недвижимости
15868,1,-3843.290778,35,среднее,1,в разводе,3,F,компаньон,0,186303.090576,высшее образование
6764,1,-450.824369,39,высшее,0,женат / замужем,0,F,госслужащий,0,207610.334887,свой автомобиль
12054,0,-2081.968853,42,среднее,1,гражданский брак,1,M,сотрудник,0,209303.513156,покупка жилой недвижимости
18734,1,-2394.089580,48,среднее,1,женат / замужем,0,F,сотрудник,0,90452.726392,на покупку своего автомобиля
11312,0,347451.466890,60,среднее,1,женат / замужем,0,F,пенсионер,0,74600.238783,операции с коммерческой недвижимостью
8869,0,371733.755640,65,среднее,1,женат / замужем,0,F,пенсионер,0,193999.840289,на покупку своего автомобиля


### Выводы по предварительному анализу

- Некачественное название столбцов [x]
- Дубликаты значений [x]
- Пропуски значений в столбцах стаж и ежемесячный доход [x]
- Отрицательные значения в столбце дети [x]
- Некоректные значения в столбце дети (20) [x]
- Отрицательные значения в столбце трудовой стаж [x]
- Столбец стаж тип данных [x]
- Некоректные данные в возрасте клиента (0)[x]
- Плавающий регистр в столбце образование [x]
- Некоректные данные в столбце пол (3 пола)

# 3 Подготовка и очистка данных

## 3.1. Индексы и названия столбцов

In [555]:
df.columns.tolist()

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

In [556]:
columns = ['number_of_children',
             'days_employed',
             'age',
             'education',
             'education_id',
             'family_status',
             'family_status_id',
             'gender',
             'income_type',
             'debt',
             'monthly_income',
             'credit_purpose']

In [557]:
# Переименовываем и проверяем столбцы
df.set_axis(columns, axis=1, inplace=True)
df.columns.tolist()

['number_of_children',
 'days_employed',
 'age',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'monthly_income',
 'credit_purpose']

## 3.2. Поиск дубликатов

In [558]:
df.duplicated().sum()

54

Как мы можем уидеть данные содержат 54 дубликата. Стоит обратить внимание и разобраться с причинами их возникновения.
Скорее всего они появились из-за ошибки правил валидации на сервере.

In [559]:
# удаляем дубликаты и сбрасываем индекс
df = df.drop_duplicates().reset_index(drop=True)

In [560]:
# Проверка 
df.duplicated().sum()

0

## 3.3. Поиск пропущенных значений

In [561]:
# разбирался с Series
null_percentage = df.isnull().sum()/len(df)*100
null_quantity = df.isnull().sum()
col_isnull = pd.concat([null_percentage, null_quantity], axis=1).sort_values(by=1, ascending=False)
col_isnull.columns = ['null_percentage','null_quantity']
col_isnull.head()

,null_percentage,null_quantity
days_employed,9.873783,2120
monthly_income,9.873783,2120
number_of_children,0.000000,0
age,0.000000,0
education,0.000000,0


Мы наблюдаем что в столбцах 'total_income' и days_employed отсутствует около 10% значений.

❗️ **Второстипенная проблема отсутствующая в задачах исследования.**

Вариант для проработки 2 сценариев и обсуждения вопроса с заказчиком:

- Зависит от ситуации в настоящий момент - много задолжностей либо мало обращений

- Заполнение отсутсвующих данных значением медианы повысит шанс получения кредита (если подставим медиану то банк берет на себя дополнительные риски). Заполнение 0 - снизит (если подставим 0 то банк лишиться потенциальных клиентов)


## 3.4. Обработка отсутствующих значений

### 3.4.1. Обработка пропущенных значений столбца 'days_employed'

In [562]:
df['days_employed'].value_counts(dropna=False).head(5)

 NaN            2120
-1645.463049       1
-6620.396473       1
-1238.560080       1
-3047.519891       1
Name: days_employed, dtype: int64

В данном столбце 2174 нулевых значения. Так как значение столбца 'days_employed' количественное, заменим пропущенные значения в столбце на значение медианы

In [563]:
#Замена
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
# Проверка замены
df.days_employed.value_counts().head(5)

-1203.369529    2121
-986.927316        1
-1893.222792       1
-4236.274243       1
-6620.396473       1
Name: days_employed, dtype: int64

### 3.4.2. Обработка пропущенных значений столбца 'monthly_income'

In [564]:
df['monthly_income'].value_counts(dropna=False).head(5)

NaN              2120
133912.272223       1
182036.676828       1
122421.963500       1
198271.837248       1
Name: monthly_income, dtype: int64

В данном столбце как и в предыдущем примере 2175 нулевых значения. Этот столбец очень важен для анализа, так как ежемесячный доход это одна из главных метрик показателя платежеспособности клиента. Так как значение столбца 'monthly_income' количественное, заменим пропущенные значения в столбце на значение медианы

In [565]:
#Замена
df['monthly_income'] = df['monthly_income'].fillna(df['monthly_income'].median())
# Проверка замены
df['monthly_income'].value_counts().head()

145017.937533    2121
112874.418757       1
104381.857170       1
182036.676828       1
122421.963500       1
Name: monthly_income, dtype: int64

## 3.5. Обработка столбцов

Для большей наглядности исследование каждого столбца буду начинать с постоения диаграммы 'ящик с усами', что позволит взглянуть на выбросы и ознакомиться с данными более глубже

### 3.5.1 Количественные признаки

- number_of_children
- days_employed => years_employed
- age
- education_id
- family_status_id
- debt
- monthly_income

#### 3.5.1.1 Дети

In [566]:
children = px.box(df, y='number_of_children', title='Children')
children.show()

In [567]:
df['number_of_children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: number_of_children, dtype: int64

График четко показал на выброс. 20 детей это перебор, как и -1. Здесь можно пойти 2 путями либо посчить эти данные нерепрезентативными и удалить, либо посчитать, что произошла ошибка при заполнении и испраить 20 на 2, а -1 на 1.
Так как дети наряду с семейным положением это самая важная метрика в данном исследовании [Наверх к цели исследования](#1), то будем производить замену а не лишаться данных.

In [568]:
df['number_of_children'] = df['number_of_children'].replace([20,-1], [2, 1])

In [569]:
# Проверяем замену
df['number_of_children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: number_of_children, dtype: int64

#### 3.5.1.2 Трудовой стаж

In [570]:
days_employed = px.box(df, y='days_employed', title='Days employed')
days_employed.show()

In [571]:
df['days_employed'].describe()

count     21471.000000
mean      56702.605042
std      135060.747564
min      -18388.949901
25%       -2522.536607
50%       -1203.369529
75%        -382.945412
max      401755.400475
Name: days_employed, dtype: float64

Сразу же бросается в глаза выброс в район 400 тыс. Даже с учетом того, что в столбце стаж отображается в днях. Люди столько не работают и даже не живут ```400000/365 = 1095``` 😦.

❗️  Пока нам эта метрика не столь важна.


Далее мы изменим отрицательные значения на положительные и изменим тип данных на целое число, а также сделаем перевод дня в год


In [572]:
# Убираем отрицательные значения переводим в года, приводим к типу целое число и дропаем дни
df['days_employed'] = df['days_employed'].abs()
df['years_employed'] = df['days_employed']/365
df['years_employed'] = df['years_employed'].astype(int)
del df['days_employed']

#### 3.5.1.3 Возраст

In [573]:
age = px.box(df, y='age', title='Age')
age.show()

In [574]:
# считаем 0 значения
df[df['age'] == 0]['age'].count()

101

Из графика и подсчета количества значений с возрастом 0 видим что таких значений 101. Здесь можно разбросать данные по категориям, но так как данная метрика не участвует в анализе пока ее не трогаем

education_id
family_status_id
debt
monthly_income

#### 3.5.1.4 Образование
Данный признак переведен из категориального в количественный

In [591]:
df['education_id'].value_counts()

1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

#### 3.5.1.5 Семейное положение
Данный признак переведен из категориального в количественный

In [593]:
df['family_status_id'].value_counts()

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

#### 3.5.1.6 Задолженность по кредиту в прошлом

In [599]:
df['debt'].value_counts()

0    19730
1     1741
Name: debt, dtype: int64

#### 3.5.1.7  Ежемесячный доход

In [595]:
monthly_income = px.box(df, y='monthly_income', title='Мonthly income')
monthly_income.show()

Из данного столбца не совсем понятна единица измерения. Максимум  более 2 млн., медиана около 150000, а минимум 20000.

In [604]:
# Посмотрим сколько людей получают в месяц более 150000
df[df['monthly_income'] > 150000].count().head(1)


number_of_children    9184
dtype: int64

#### 3.5.1.2 Образование
Данный признак переведен из категориального в количественный

In [583]:
# Посмотрим на уникальные значения
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [589]:
# Приводим к нижнему регистру и проверяем
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Приступим к изучению категориальных признаков

### 3.5.2 Категориальные признаки

- education
- family_status
- gender
- income_type	
- credit_purpose

#### 3.5.1.2 Образование

In [607]:
df

,number_of_children,age,education,education_id,family_status,family_status_id,gender,income_type,debt,monthly_income,credit_purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,12
21467,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,942
21468,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,5
21469,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,8


In [576]:
# df['education'] = df['education'].str.lower()

In [577]:
# df['education'].value_counts()

**Вывод:**
На этапе предобработки в данных обнаружились не только пропуски но дубликаты. Удаление дубликатов и замена пропусков позволит провести анализ точнее. Поскольку сведения о ежемесячном доходе и трудовом стаже важны для анализа, не просто удаляем все пропущенные значения, заполняем их значением медианы. 

На графике 'days_employed' также имеется выброс значений в **количестве 3445 штук**. Это влияет на сдвиг среднего поэтому при подстановке используем медиану. 

In [618]:
median_values = pd.pivot_table(
    df,
    index=['education', 'income_type'],
    values=['years_employed', 'monthly_income'],
    aggfunc=np.median
)

median_values

monthly_income  years_employed
education           income_type                                    
высшее              безработный       202722.511368          1083.0
                    госслужащий       158384.910405             6.0
                    компаньон         191266.902914             3.0
                    пенсионер         145017.937533           992.5
                    предприниматель   322090.541240             2.0
                    сотрудник         155135.663004             3.0
                    студент            98201.625314             1.0
начальное           госслужащий       145017.937533             4.5
                    компаньон         140908.421338             3.0
                    пенсионер         107398.699119           981.0
                    сотрудник         131629.331952             3.0
неоконченное высшее госслужащий       156266.846399             3.5
                    компаньон         169674.017186             2.0
                    пенсионер         129742.715263           991.0
                    сотрудник         145017.937533             3.0
среднее             безработный        59956.991984           924.0
                    в декрете          53829.130729             9.0
                    госслужащий       144351.310834             6.5
                    компаньон         149106.732653             3.0
                    пенсионер         124675.940120           987.0
                    сотрудник         144970.433937             3.0
ученая степень      госслужащий       111392.231107            16.0
                    пенсионер         177088.845999           977.0
                    сотрудник         198570.757322             6.0

In [617]:
median_values.to_csv('pivot.csv', encoding='utf-8')